# 跟随以下提示复现项目：

在开始浮现之前你首先要了解如何更改配置与项目结构，项目结构如下

项目目录（推荐为 /home/xilinx/jupyter_notebooks/SGSAM/ ）

        ├── experiments/                # experiments目录
        │   ├── params.npz              # final recon需要的参数文件
        │
        ├── utils/                      # utils目录
        │   ├── common_helpers.py
        │   ├── ip_helpers.py
        │   ├── rasterization.py
        │   ├── rasterize_helpers.py
        │   ├── recon_helpers.py
        │   └── slam_helpers.py
        │
        ├── configs.py        # <---------配置文件，可配置相机拍摄帧数，反向传播迭代数，学习率等参数
        ├── iphone_demo.ipynb           # 当前文件
        ├── final_recon.ipynb           # final_recon 文件
        ├── first_frame.pkl             # 备用方案示例帧 
        └── output_img.jpg              # 输出的jpg格式图像


1. 要首你需要AMD官方PYNQ镜像来运行Jupyter NoteBook

2. 你需将我们提供的vitis_design.bit和vitis_design.hwh文件放置在/home/xilinx/pynq/overlays目录下，例如：在其中创建my_project目录将.bit和.hwh文件复制到目录中，这样就不用更改下方单元格中的代码

3. 运行下方单元各即可加载overlay


In [ ]:
import pynq
from pynq import allocate

ol = pynq.Overlay('my_project/vitis_design.bit')
ol?

4. 获取依赖包，需要注意的是，原生Pynq镜像不包含torch这个库，在Pynq控制台终端执行以下命令来安装torch

        source /etc/profile.d/pynq_venv.sh
        pip install --upgrade numpy==1.23.5
        source /etc/profile.d/pynq_venv.sh
        pip install --user torch==1.13.1

In [ ]:
import cv2
import time
import pickle
import struct
import asyncio
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm
from configs import get_config
from utils.ip_helpers import *
from utils.slam_helpers import *
from utils.rasterization import *
from utils.recon_helpers import *
from utils.common_helpers import *
from utils.rasterize_helpers import *

5. 安装cyclonedds   ( 来源：https://github.com/eclipse-cyclonedds/cyclonedds )

    在Pynq终端输入：

        git clone https://github.com/eclipse-cyclonedds/cyclonedds
        cd cyclonedds
        git checkout 0.10.5
        mkdir build install && cd build
        cmake .. -DCMAKE_INSTALL_PREFIX=../install
        cmake --build . --config RelWithDebInfo --target install
        cd ..
        export CYCLONEDDS_HOME="$(pwd)/install"
        pip3 install cyclonedds==0.10.5

In [ ]:
import cyclonedds.idl as idl
import cyclonedds.idl.annotations as annotate
import cyclonedds.idl.types as types
from dataclasses import dataclass
from cyclonedds.domain import DomainParticipant, Domain
from cyclonedds.core import Qos, Policy
from cyclonedds.sub import DataReader
from cyclonedds.topic import Topic
from cyclonedds.util import duration

@dataclass
@annotate.final
@annotate.autoid("sequential")
class SplatCaptureFrame(idl.IdlStruct, typename="SplatCaptureData.SplatCaptureFrame"):
    id: types.uint32
    annotate.key("id")
    timestamp: types.float64
    fl_x: types.float32
    fl_y: types.float32
    cx: types.float32
    cy: types.float32
    transform_matrix: types.array[types.float32, 16]
    width: types.uint32
    height: types.uint32
    image: types.sequence[types.uint8]
    has_depth: bool
    depth_width: types.uint32
    depth_height: types.uint32
    depth_scale: types.float32
    depth_image: types.sequence[types.uint8]

6. 在安装完成cyclonedds后，在控制台输入

        sudo sysctl -w net.core.rmem_max=2147483647

        sudo sysctl -w net.core.wmem_max=2147483647

完成对网络读写容量的配置，

接下来你需要根据你为zcu104板卡分配的ip地址替换下方单元格中的192.168.xxx.xxx

In [ ]:
dds_config = """<?xml version="1.0" encoding="UTF-8" ?> \
<CycloneDDS xmlns="https://cdds.io/config" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="https://cdds.io/config https://raw.githubusercontent.com/eclipse-cyclonedds/cyclonedds/master/etc/cyclonedds.xsd"> \
    <Domain id="any"> \
        <General> \
            <NetworkInterfaceAddress>192.168.xxx.xxx</NetworkInterfaceAddress> \
        </General> \
        <Internal> \
            <MinimumSocketReceiveBufferSize>10MB</MinimumSocketReceiveBufferSize> \
        </Internal> \
        <Tracing> \
            <Verbosity>config</Verbosity> \
            <OutputFile>stdout</OutputFile> \
        </Tracing> \
    </Domain> \
</CycloneDDS> \
"""

7. 要运行下方单元格内容，需要将支持 $1920 \times 1200$ 的DP显示屏连接到zcu104的DisplayPort上，或通过主动式DP转HDMI转换器连接同样规格的HDMI显示屏，并为显示屏通电

    如果无此配置的显示屏则需要修改VideoMode(1920, 1200, 24)

    如果不想使用显示屏可以删除或备注下面4行

        from pynq.lib.video import *
        displayport = DisplayPort()
        displayport.configure(VideoMode(1920, 1200, 24), PIXEL_RGB)
        displayport.start()

    无cycloneDDS可以删除或备注下面7行

        # Setup DDS
        domain = Domain(domain_id=0, config=dds_config)
        participant = DomainParticipant()
        qos = Qos(Policy.Reliability.Reliable(
            max_blocking_time=duration(seconds=1)))
        topic = Topic(participant, "Frames", SplatCaptureFrame, qos=qos)
        reader = DataReader(participant, topic)

In [ ]:
from pynq.lib.video import *
displayport = DisplayPort()
displayport.configure(VideoMode(1920, 1200, 24), PIXEL_RGB)
displayport.start()

config = get_config()

# Set Seed
seed_everything(seed=config['seed'])

# Setup DDS
domain = Domain(domain_id=0, config=dds_config)
participant = DomainParticipant()
qos = Qos(Policy.Reliability.Reliable(
    max_blocking_time=duration(seconds=1)))
topic = Topic(participant, "Frames", SplatCaptureFrame, qos=qos)
reader = DataReader(participant, topic)

# Create Results Directory and Copy Config
results_dir = os.path.join(
    config["workdir"], config["run_name"]
)
os.makedirs(results_dir, exist_ok=True)

8. 运行下方单元格来进行建图，在iPhone14以上带TOF传感器的机型上运行 **NeRF Capture** app ，在Waiting for frames...字样出现后拍照，注意手机与板卡应当处在同一局域网下

    如果cycloneDDS安装失败，或缺少**NeRF Capture** app ，可以将单元格中的

        sample = reader.read_next()

    替换为

        with open('first_frame.pkl', 'rb') as f:
            sample = pickle.load(f)
    
    确保first_frame.pkl与该NoteBook在同意目录下

    如果之前决定不使用显示屏，将下方单元格99~117行备注或删除，具体为以下内容：

        im = im.clone().detach().numpy().reshape(360, 480, 3) * 255
        im[im > 255] = 255
        im[im < 0] = 0
        img_uint8 = im.astype(np.uint8)
        img_RGB = cv2.cvtColor(img_uint8, cv2.COLOR_BGR2RGB)
        # 获取原始图像尺寸
        orig_height, orig_width = img_RGB.shape[:2]
        orig_height *= 3
        orig_width *= 3
        img_RGB = cv2.resize(img_RGB, (orig_width,orig_height))
        # 创建黑色背景帧
        frame = displayport.newframe()
        frame[:] = 0  # 填充黑色
        # 计算居中位置
        start_x = (1920 - orig_width) // 2
        start_y = (1200 - orig_height) // 2
        # 将图像复制到帧的中央位置
        frame[start_y:start_y+orig_height, start_x:start_x+orig_width] = img_RGB
        displayport.writeframe(frame)


In [ ]:
print("Waiting for frames...")
total_frames = 0 # Total frames received
time_idx = total_frames
num_frames = config["num_frames"] # Total frames desired

# Init Variables to keep track of ARkit poses and runtimes
mapping_iter_time_sum = 0
mapping_iter_time_count = 0
mapping_frame_time_sum = 0
mapping_frame_time_count = 0
P = torch.tensor(
    [[1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, 1]]).float()

ip = ol.rasterization_1

# Start DDS Loop
while True:
    sample = reader.read_next()
    
    if sample:
        print(f"{total_frames + 1}/{num_frames} frames received")

        # RGB & Depth
        image = np.asarray(sample.image, dtype=np.uint8).reshape((sample.height, sample.width, 3))
        if sample.has_depth:
            curr_depth = np.asarray(sample.depth_image, dtype=np.uint8).view(
                dtype=np.float32).reshape((sample.depth_height, sample.depth_width))
        else:
            print("No Depth Image Received. Please make sure that the NeRFCapture App \
                  mentions Depth Supported on the top right corner. Skipping Frame...")
            continue

        # ARKit Poses for saving dataset & Convert ARKit Pose to GradSLAM format
        X_WV = np.asarray(sample.transform_matrix, dtype=np.float32).reshape((4, 4)).T
        gt_pose = torch.from_numpy(X_WV).float()
        gt_pose = P @ gt_pose @ P.T
        gt_pose = gt_pose.unsqueeze(0)
        gt_w2c = torch.linalg.inv(gt_pose[0])

        # Initialize Mapping Resolution Data
        color = cv2.resize(image, dsize=(
            config['data']['image_width'], config['data']['image_height']), interpolation=cv2.INTER_LINEAR)
        depth = cv2.resize(curr_depth, dsize=(
                config['data']['image_width'], config['data']['image_height']), interpolation=cv2.INTER_NEAREST)
        depth = np.expand_dims(depth, -1)

        color = torch.from_numpy(color).float()
        depth = torch.from_numpy(depth).float()
        color /= 255

        intrinsics = torch.tensor([[sample.fl_x, 0, sample.cx], [0, sample.fl_y, sample.cy], [0, 0, 1]]).float()
        intrinsics = intrinsics / config['data']['downscale_factor']
        intrinsics[2, 2] = 1.0
        mask = (depth > 0) # Mask out invalid depth values
        thresholds = method_custom_equal_height(depth[mask], 256)
        cam = setup_camera(color.shape[1], color.shape[0], thresholds, intrinsics.cpu().numpy(), gt_w2c.cpu().numpy())

        # Initialize Params for first time step
        if time_idx == 0:
            mask = mask.reshape(-1)
            init_pt_cld, mean3_dist = get_pointcloud(color, depth, intrinsics, gt_w2c,
                                                        mask=mask, compute_mean_sq_dist=True,
                                                        mean_sq_dist_method=config['mean_sq_dist_method'])
            params, variables = initialize_params(init_pt_cld, num_frames, mean3_dist)
            variables['scene_radius'] = torch.max(depth)/config['scene_radius_depth_ratio']

        # Initialize Mapping & Tracking for current frame
        iter_time_idx = time_idx
        curr_data = {'cam': cam, 'im': color, 'depth': depth, 'id': iter_time_idx, 'curr_w2c': gt_w2c, 'intrinsics':intrinsics }
        num_iters_mapping = config['mapping']['num_iters']

        # Densification
        if time_idx == 0 or (time_idx+1) % config['map_every'] == 0:
            if config['mapping']['add_new_gaussians'] and time_idx > 0:
                params, variables = add_new_gaussians(ip,params, variables, curr_data,
                                                    config['mapping']['sil_thres'], time_idx,
                                                    config['mean_sq_dist_method'])

        # Reset Optimizer & Learning Rates for Full Map Optimization
        optimizer = initialize_optimizer(params, config['mapping']['lrs'])

        # Mapping
        for iter in range(num_iters_mapping):
            iter_start_time = time.time()

            # Loss for current frame
            loss, variables, losses, im = get_loss(ip, params, curr_data, variables, time_idx, config['mapping']['loss_weights'],
                                            config['mapping']['use_sil_for_loss'], config['mapping']['sil_thres'],
                                            config['mapping']['use_l1'], config['mapping']['ignore_outlier_depth_loss'])
            
            get_loss_time = time.time()
            # Backprop
            loss.backward()
            backward_time = time.time()
            
            im = im.clone().detach().numpy().reshape(360, 480, 3) * 255
            im[im > 255] = 255
            im[im < 0] = 0
            img_uint8 = im.astype(np.uint8)
            img_RGB = cv2.cvtColor(img_uint8, cv2.COLOR_BGR2RGB)
            # 获取原始图像尺寸
            orig_height, orig_width = img_RGB.shape[:2]
            orig_height *= 3
            orig_width *= 3
            img_RGB = cv2.resize(img_RGB, (orig_width,orig_height))
            # 创建黑色背景帧
            frame = displayport.newframe()
            frame[:] = 0  # 填充黑色
            # 计算居中位置
            start_x = (1920 - orig_width) // 2
            start_y = (1200 - orig_height) // 2
            # 将图像复制到帧的中央位置
            frame[start_y:start_y+orig_height, start_x:start_x+orig_width] = img_RGB
            displayport.writeframe(frame)

            with torch.no_grad():
                # Optimizer Update
                optimizer.step() 
                optimizer.zero_grad(set_to_none=True)
                
            # Update the runtime numbers
            iter_end_time = time.time()
            mapping_iter_time_sum += iter_end_time - iter_start_time
            mapping_iter_time_count += 1

        # Update frame count at end
        if total_frames == num_frames - 1:
            break
        total_frames += 1
        time_idx = total_frames

9. 运行下方单元格将保存经过方向传播优化参数后的渲染结果，视角与最后一帧图片相同

In [ ]:
img_bgr = cv2.cvtColor(img_uint8, cv2.COLOR_RGB2BGR)
cv2.imwrite('output_image.jpg', img_bgr)

10. 运行下方单元格保存点云供final recon使用

In [ ]:
import os
import sys
print(curr_data['intrinsics'])

params_to_save = {
    'means3D': params['means3D'].detach().numpy(),
    'rgb_colors': params['rgb_colors'].detach().numpy(),
    'logit_opacities': params['logit_opacities'].detach().numpy(),
    'scales': torch.exp(params['log_scales']).detach().numpy(),
    'width': 480,
    'height': 360,
    'intrinsics': curr_data['intrinsics'].detach().numpy(),
}
_BASE_DIR = os.getcwd()
sys.path.insert(0, _BASE_DIR)
results_dir = os.path.join(_BASE_DIR, "experiments")
scene_path = os.path.join(results_dir, "params.npz")
np.savez(scene_path, **params_to_save)